# Agrupamento K-Means - Experimento

Este é um componente que treina um modelo K-Means usando [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html). <br>
Scikit-learn é uma biblioteca open source de machine learning que suporta apredizado supervisionado e não supervisionado. Também provê várias ferramentas para ajustes de modelos, pré-processamento de dados, seleção e avaliação de modelos, além de outras funcionalidades.

Este notebook apresenta:
- como usar o [SDK](https://platiagro.github.io/sdk/) para carregar datasets, salvar modelos e outros artefatos.
- como declarar parâmetros e usá-los para criar componentes reutilizáveis.

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
O parâmetro `dataset` identifica os conjuntos de dados. Você pode importar arquivos de dataset com o botão  na barra de ferramentas.

In [ ]:
# parâmetros
dataset = "iris" #@param {type:"string"}
n_clusters = 3 #@param {type:"integer"}
n_init = 10 #@param {type:"integer"}
max_iter = 300 #@param {type:"integer"}
algorithm = "auto" #@param {type:"string"}

## Acesso ao conjunto de dados

Utiliza a função `load_dataset` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para carregar conjuntos de dados.
O tipo da variável retornada depende do arquivo de origem:
- [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
- [Binary IO stream](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [ ]:
from platiagro import load_dataset

df = load_dataset(name=dataset)

## Acesso aos metadados do conjunto de dados

Utiliza a função `stat_dataset` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para carregar metadados. <br>
Por exemplo, arquivos CSV possuem `metadata['featuretypes']` para cada coluna no conjunto de dados (ex: categorical, numerical, or datetime).

In [ ]:
import numpy as np
from platiagro import stat_dataset

metadata = stat_dataset(name=dataset)
featuretypes = metadata["featuretypes"]
featuretypes = np.array(featuretypes)
columns = df.columns.to_numpy()

## Configuração das features

In [ ]:
from platiagro.featuretypes import NUMERICAL

# Selects the indexes of numerical and non-numerical features
numerical_indexes = np.where(featuretypes == NUMERICAL)[0]
non_numerical_indexes = np.where(~(featuretypes == NUMERICAL))[0]

# After the step handle_missing_values, 
# numerical features are grouped in the beggining of the array
numerical_indexes_after_handle_missing_values = \
    np.arange(len(numerical_indexes))
non_numerical_indexes_after_handle_missing_values = \
    np.arange(len(numerical_indexes), len(featuretypes))

## Treina modelo usando sklearn.cluster.KMeans

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from category_encoders.ordinal import OrdinalEncoder

pipeline = Pipeline(steps=[
            ("handle_missing_values",
             ColumnTransformer(
                [("imputer_mean", SimpleImputer(strategy="mean"), numerical_indexes),
                 ("imputer_mode", SimpleImputer(strategy="most_frequent"), non_numerical_indexes)],
                 remainder="drop")),
            ("handle_categorical_features",
             ColumnTransformer(
                 [("feature_encoder", OrdinalEncoder(), non_numerical_indexes_after_handle_missing_values)],
                 remainder="passthrough")),
            ("estimator", KMeans(n_clusters=n_clusters,
                                    n_init=n_init,
                                    max_iter=max_iter,
                                    algorithm=algorithm))
])

X = pipeline.fit_transform(df)

## Avalia desempenho

No caso do K-Means, podemos medir o desempenho pela pontuação da silhueta.

In [ ]:
import numpy as np
import pandas as pd
import sklearn.metrics as metrics

labels = pipeline.named_steps.estimator.labels_
score = metrics.silhouette_score(X, labels, metric="euclidean")

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X_std = StandardScaler().fit_transform(X.copy())
pca = PCA(n_components=2)
reduced = pca.fit_transform(X_std)

X_pca = pd.DataFrame(reduced, columns=["X", "Y"])

X_pca["Cluster"] = labels

In [ ]:
from platiagro import save_figure
import matplotlib.pyplot as plt
import seaborn as sns

ax = sns.scatterplot(x="X", y="Y", hue="Cluster", data=X_pca)

ax.set_title("PCA Graph", {"fontweight": "bold"})

plt.grid(True)

save_figure(figure=plt.gcf())

## Salva métricas

Utiliza a função `save_metrics` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para salvar métricas.

In [ ]:
from platiagro import save_metrics

save_metrics(silhouette_score=score)

## Salva conjunto de dados

In [ ]:
from platiagro import save_dataset

new_columns = list("Clusters")

df_clusters = df.copy()
df_clusters["Clusters"] = labels
save_dataset(name=dataset, df=df_clusters)

## Salva modelo e outros artefatos

Utiliza a função `save_model` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para salvar modelos e outros artefatos.<br>
Essa função torna estes artefatos disponíveis para o notebook de implantação.

In [ ]:
from platiagro import save_model

save_model(pipeline=pipeline, columns=columns, new_columns=new_columns)